In [2]:
import pandas as pd
dataset_name = 'en'
step_turn = 'step2'
seg_df = pd.read_excel('../step1-job_task_extraction/results/en/en-job_re_description.xlsx')
origin_df = pd.read_csv('../data/en/job_description.csv', sep='\t')
df_map = {'jd_seg':seg_df, 'jd_ori': origin_df}

In [7]:
import json
task_kind = 'task'
task_map = json.load(open(f'data/en/{step_turn}/{task_kind}.json', 'r'))
task_rule_map = {i:'|'.join(v) for i,v in task_map.items()}
task_rule_map = {i:f"{v}" for i,v in task_rule_map.items()}
task_rule_map

{'Read materials to determine needed actions.': 'Data Analysis|Research and Analysis',
 'Investigate legal issues': 'Legal Research|Litigation Support|Legal Opinions|Legal Documents',
 'Research product safety': 'safety document|safety procedures|food safety',
 'Monitor equipment operation to ensure proper functioning.': 'Equipment Monitoring|Preventive Maintenance',
 'Mark materials or objects for identification.': 'Labeling|Tagging|Identification',
 'Test quality of materials or finished products.': 'Quality Assurance Testing|Material Quality Evaluation|Product Quality Control|Final Product Testing',
 'Inspect food products.': 'food safety|Food Quality',
 'Test electrical circuits or components for proper functioning.': 'Circuit|Electrical Testing|Electrical components',
 'Calculate dimensions of workpieces, products, or equipment.': 'dimensions|Geometric|Blueprint',
 'Evaluate student work.': 'Curriculum Development|Student Evaluation|Student Engagement|teaching student|student work

In [42]:
df_map = {'jd_seg':seg_df, 'jd_ori': origin_df}
import re
import os
import copy
from tqdm import tqdm
tqdm.pandas()
cal_num = {}
def split_sentence_by_uppercase(sentence):
    # 在大写字母前插入空格
    return re.sub(r'([A-Z])', r' \1', sentence).strip()

if not os.path.exists(f"outputs/{step_turn}"):
    os.makedirs(f"outputs/{step_turn}")
for df_type, df_bp in df_map.items():
    all_df = []
    cal_num[df_type] = {}
    if os.path.exists(f'data/en/{step_turn}/{df_type}.parquet'):
        df = pd.read_parquet(f'data/en/{step_turn}/{df_type}.parquet')
    else:
        df = copy.deepcopy(df_bp)
        if df_type == 'jd_seg':
            df['job_description'] = df['task']
        df['job_description'] = df.apply(lambda x: [line for line in re.findall(r'[A-Z][^A-Z]*', split_sentence_by_uppercase(x['job_description']))] if type(x['job_description']) == type('') else [], axis=1)
        df = df.explode('job_description')
        df = df[~df['job_description'].isna()]
        df['len'] = df['job_description'].apply(lambda x: len(x.split(' ')))
        df = df[(df['len']>6) & (df['len']<20)]
        df = df.drop_duplicates('job_description')
        df.to_parquet(f'data/en/{step_turn}/{df_type}.parquet', index=None)

    for task, task_rule in tqdm(task_rule_map.items(), total=len(task_rule_map)):
        if len(task_rule) == 0:
            continue            
        df[task] = df['job_description'].apply(lambda x: len(re.findall(task_rule, x, re.IGNORECASE)) > 0)
        sub_df = df[df[task]][['job_title', 'job_description']]
        print(df_type, task, len(sub_df))
        cal_num[df_type][task] = len(sub_df)
        # sub_df = sub_df.sample(frac=1.0)
        sub_df['task'] = task
        all_df.append(sub_df)
    all_df = pd.concat(all_df)[['task', 'job_title', 'job_description']].reset_index(drop=True)
    all_df.to_excel(f'outputs/{step_turn}/key_extraction_{df_type}-{task_kind}.xlsx', index=None)
cal_num = pd.DataFrame(cal_num)
cal_num

  6%|▌         | 2/35 [00:00<00:02, 14.74it/s]

jd_seg Read materials to determine needed actions. 63
jd_seg Investigate legal issues 42
jd_seg Research product safety 66


 11%|█▏        | 4/35 [00:00<00:02, 10.47it/s]

jd_seg Monitor equipment operation to ensure proper functioning. 40
jd_seg Mark materials or objects for identification. 89


 20%|██        | 7/35 [00:00<00:03,  9.00it/s]

...

 97%|█████████▋| 34/35 [02:59<00:04,  4.88s/it]

jd_ori Arrange insurance coverage. 137


100%|██████████| 35/35 [03:03<00:00,  5.24s/it]

jd_ori Prescribe medications. 280


,jd_seg,jd_ori
Read materials to determine needed actions.,63,2389
Investigate legal issues,42,760
Research product safety,66,1178
Monitor equipment operation to ensure proper functioning.,40,607
Mark materials or objects for identification.,89,2024
Test quality of materials or finished products.,3,43
Inspect food products.,23,644
Test electrical circuits or components for proper functioning.,31,1028
"Calculate dimensions of workpieces, products, or equipment.",46,1412
Evaluate student work.,18,289


In [8]:
cal_num = {}
for df_type, df_bp in df_map.items():
    df = pd.read_excel(f'outputs/{step_turn}/key_extraction_{df_type}.xlsx')
    cal_num[df_type] = {}
    for task, task_rule in task_rule_map.items():
        if len(task_rule) == 0:
            continue   
        cal_num[df_type][task] = len(df[df['task']==task])
cal_num = pd.DataFrame(cal_num)
cal_num

,jd_seg,jd_ori
Read materials to determine needed actions.,63,2389
Investigate legal issues,42,760
Research product safety,66,1178
Monitor equipment operation to ensure proper functioning.,40,607
Mark materials or objects for identification.,89,2024
Test quality of materials or finished products.,3,43
Inspect food products.,23,644
Test electrical circuits or components for proper functioning.,31,1028
"Calculate dimensions of workpieces, products, or equipment.",46,1412
Evaluate student work.,18,289


In [9]:
import pandas as pd
cal_num = {}
final_df = None
for df_type, df_bp in df_map.items():
    df = pd.read_excel(f'outputs/{step_turn}/key_extraction_{df_type}.xlsx')
    if final_df is None:
        final_df = df
    else:
        final_df_group = final_df.groupby('task').agg(list)
        final_df_group['len'] = final_df_group['job_description'].apply(len)
        df_group = df.groupby('task').agg(list)
        df_group['len'] = df_group['job_description'].apply(len)
        final_df = pd.concat(final_df)
cal_num = pd.DataFrame(cal_num)
cal_num

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [ ]:
# tqdm.pandas()
# all_df = pd.read_excel('outputs/key_extraction_fromjd.xlsx')
# all_df['extract_task'] = all_df.progress_apply(lambda x: [line for line in re.findall(r'[A-Z][^A-Z]*', split_sentence_by_uppercase(x['job_description'])) if re.search(rules[task_list.index(x['task'])], line, re.IGNORECASE)] if type(x['job_description']) == type('') else [], axis=1)
# all_df.to_excel('outputs/key_extraction_fromjd_annotated.xlsx', index=None)
# all_df_ch = pd.read_excel('outputs/key_extraction-ch.xlsx')
# final_df = pd.concat([all_df, all_df_ch], axis=1)
# final_df.to_excel('outputs/key_extraction-merge.xlsx', index=None)
# final_df